In [5]:
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

In [6]:
class_count = 2

def generate_two_classes_data(feature_count: int, object_count: int):
    """
    Generates two classes: the first class has features near 0, the second – near 1.
    :returns features (with bias=1 feature), answers
    """
    features = np.concatenate((
        np.random.normal(
            np.concatenate((
                np.zeros([object_count // 2, feature_count], dtype=np.float64),
                np.ones([object_count - object_count // 2, feature_count], dtype=np.float64)
            )),
            scale=0.1
        ),
        np.ones([object_count, 1], dtype=np.float64)
    ), axis=1)

    answers = np.concatenate((
        np.concatenate((
            np.zeros([object_count // 2, 1], dtype=np.float64),
            np.ones([object_count - object_count // 2, 1], dtype=np.float64)
        )),
        np.concatenate((
            np.ones([object_count // 2, 1], dtype=np.float64),
            np.zeros([object_count - object_count // 2, 1], dtype=np.float64)
        ))
    ), axis=1)

    return features, answers

f, a = generate_two_classes_data(2, 3)
print(f)
print(a)

[[0.08725635 0.13529509 1.        ]
 [1.02984784 1.08944507 1.        ]
 [0.99803821 0.93320117 1.        ]]
[[0. 1.]
 [1. 0.]
 [1. 0.]]


In [7]:
np.random.seed(0)

times = 10

stats = pd.DataFrame(columns=['feature_count', 'g_delta', 'h_delta'])

for feature_count in (10**2, 10**3, 10**4):
# for feature_count in (10**2, 10**3, 10**4, 10**5, 10**6):
    object_count = 200

    features, answers = generate_two_classes_data(feature_count=feature_count, object_count=object_count)

    x = tf.placeholder(tf.float64, [None, feature_count + 1])
    y = tf.placeholder(tf.float64, [None, class_count])

    W = tf.Variable(tf.zeros([feature_count + 1, class_count], tf.float64))

    predictions = tf.nn.softmax(tf.matmul(x, W))
    cost = tf.nn.sigmoid_cross_entropy_with_logits(logits=predictions, labels=y)

    g = tf.gradients(cost, W)[0]
    h = tf.hessians(cost, W)[0]
    # h_vec_product = tf.matmul(h, W)  # todo: fix weights

    init = tf.global_variables_initializer()

    g_deltas = []
    h_deltas = []

    for i in range(times):
        with tf.Session() as sess:
            sess.run(init)

            start = datetime.now().timestamp()
            sess.run(g, feed_dict={x: features, y: answers})
            g_deltas.append(datetime.now().timestamp() - start)

            start = datetime.now().timestamp()
            sess.run(h, feed_dict={x: features, y: answers})
            h_deltas.append(datetime.now().timestamp() - start)

    g_delta = sum(g_deltas) / len(g_deltas)
    h_delta = sum(h_deltas) / len(h_deltas)
    print(f"Features: {feature_count}, g delta: {g_delta}, h delta: {h_delta}")
    stats.loc[len(stats)] = [feature_count, g_delta, h_delta]

Features: 100, g delta: 0.06709966659545899, h delta: 0.2671994924545288
Features: 1000, g delta: 0.07729930877685547, h delta: 1.6330993890762329
Features: 10000, g delta: 0.09079964160919189, h delta: 15.082432126998901


In [8]:
stats

,feature_count,g_delta,h_delta
0,100.0,0.067100,0.267199
1,1000.0,0.077299,1.633099
2,10000.0,0.090800,15.082432
